In [4]:
# %pip install --upgrade --quiet  nest_asyncio



Note: you may need to restart the kernel to use updated packages.


In [1]:
from langchain_community.document_loaders.web_base import WebBaseLoader
from langchain_core.prompts.chat import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.chat_models.ollama import ChatOllama
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores.faiss import FAISS
from langchain_community.vectorstores.chroma import Chroma
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.chains.combine_documents import create_stuff_documents_chain
import nest_asyncio
nest_asyncio.apply()

In [18]:
url = ['https://www.jellyfishtechnologies.com/','https://www.jellyfishtechnologies.com/about/','https://www.jellyfishtechnologies.com/software-development-services/','https://jellyfishtechnologies.com/data-analytics-services/','https://www.jellyfishtechnologies.com/cloud-consulting-services/','https://www.jellyfishtechnologies.com/application-modernization-services/','https://www.jellyfishtechnologies.com/ai-development-services/','https://www.jellyfishtechnologies.com/devops-services/']
loader = WebBaseLoader(url)
loader.requests_per_second=1
documents = loader.load()
documents

[Document(page_content="\n\n\n\n\n\nJellyfish Technologies: Software Development Company\n\n\n\n\n\n\n \n\n\n\n\n\n  \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n \n\n\n \n\n \n\n\n\n\n\n\t\t\t\t\t\tServices\t\t\t\t\t \n\n\n Close Services\n\n\n Open Services\n\n\n\n\n\n\t\t\t\t\t\tIndustries\t\t\t\t\t \n\n\n Close Industries\n\n\n Open Industries\n\n\n\n\n\n\t\t\t\t\t\tCase Studies\t\t\t\t\t \n\n\n\n\t\t\t\t\t\tAbout Us\t\t\t\t\t \n\n\n\n\n\n\n\n \nSoftware Development\n         Data Analytics\n                 API Integration\nMobile App Development\n   DevOps Services\nStaff Augmentation\n      Application Modernization\n       Cloud Consulting\nSoftware Outsourcing\n  CRM Development\n AI Development\nUI/UX Services\n \n\nSoftware Development\n         Data Analytics\n                 API Integration\nMobile App Development\n   DevOps Services\nStaff Augmentation\n      Application Modernization\n       Cloud Consulting\nSoftware Outsourcing\n

In [19]:
textSplitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=100)
docs = textSplitter.split_documents(documents)

In [20]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2',
                                       model_kwargs={'device': 'cpu'})

In [21]:
db = FAISS.from_documents(documents,embeddings)

In [12]:
retriever = db.as_retriever(k=5)

In [7]:
llm = ChatOllama(model='llama2',temperature=0.5,top_k=2,top_p=0.8)

In [23]:
message_history = ChatMessageHistory()

In [49]:
def chat(input):
    prompt = ChatPromptTemplate.from_messages(
        [
            ('system','You are Sriya. Your task to give information to the clients about jellyfish technologies PVT. LTD. and its offering on the bases of below context:\n\n{context}'),
            MessagesPlaceholder(variable_name="message_history"),
        ]
    )

    document_chain = create_stuff_documents_chain(llm,prompt)

    message_history.add_user_message(input)
    context = retriever.invoke(input)
    output = document_chain.invoke({'context':context,'message_history':message_history.messages})
    message_history.add_ai_message(output)
    return output

    


In [53]:
question = input()
output = chat(question)
print(output)

[INST: <<SYS>><</SYS>>]

The Chief Information Officer (CIO) of Jellyfish Technologies is [Name]. As the CIO, [Name] is responsible for leading the company's technology strategy and overseeing the development and implementation of innovative solutions that meet the unique needs of their clients across various industries.

[Name] brings a wealth of experience to the role, having held leadership positions in some of the world's top technology companies. They are known for their ability to drive business growth through technology and have a proven track record of delivering successful projects on time and within budget.

Some of [Name]'s key responsibilities as CIO of Jellyfish Technologies include:

1. Developing and implementing the company's technology strategy in alignment with its overall business goals.
2. Leading the company's software development and delivery teams, ensuring that projects are delivered on time and within budget.
3. Identifying and evaluating new technologies and t